# North Sierra Nevada Mountain Weather Analysis

This is a report on the historical analysis of weather patterns in north of Sierra Nevada Mountain(an area close to Lake Tahoe, between eastern California and Nevada).

The data we will use here comes from [NOAA](https://www.ncdc.noaa.gov/). Specifically, it was downloaded from This [FTP site](ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/).

We focused on six measurements:
* **TMIN, TMAX:** the daily minimum and maximum temperature.
* **TOBS:** The average temperature for each day.
* **PRCP:** Daily Percipitation (in mm)
* **SNOW:** Daily snowfall (in mm)
* **SNWD:** The depth of accumulated snow.

## Sanity-check: comparison with outside sources

<p>We start by comparing some of the general statistics with graphs that we obtained for city 'Truckee'(Longitude: -120.173, Latitude: 39.333) which is kind of in center of majority stations in the area under study. The graph is from a site called <a href="http://www.usclimatedata.com/climate/truckee/california/united-states/usca1163" target="_blank">US Climate Data</a>. It shows the daily minimum and maximum temperatures for each month, as well as the total precipitation for each month.</p>

<p>&nbsp;</p>

<p><img alt="Climate_of_Truckee_California_year_average.png" src="report_figures/Climate_of_Truckee_California_year_average.png" /></p>

<p>&nbsp;</p>

<p>We see that the min and max daily&nbsp;temperature agree with the ones we got from our data, both are in Fahrenheit. The minor difference in min temperature curve could be explained by our data is averaged by many locations and several stations are far east in Nevada could increase the average. And the percipitation looks quite close if we translate our data from mm to inch and aggregate the amount of days to months.</p>

<p>&nbsp;</p>

<p><img alt="TMIN_TMAX_PRCP.png" src="report_figures/TMIN_TMAX_PRCP.png"/></p>

## Goal of analysis
* This study and report tries to look for statistical significance from weather data.
* Then try to understand the intuition behind these significance corresponding to geographical information.
* And try to group stations into different clusters which show similar weather pattern.
* (Not done)For each cluster, build model/predict each weather feature for any location in this area according to its geographical information.

## Overview of statistics
### mean and standard deviation
![stat_mean_std.png](report_figures/stat_mean_std.png)
As we can see from above yearly plotting, the 3 temperature measurements look quite similar with just some up and down shifting, and the curves are smooth which means temperature change is stable on daily basis.

Snow and percipitation curves are very fluctuated on daily basis due to their nature. And snow depth is comparatively less noisy because it's decaying integral of SNOW.

## PCA analysis

For each of the six measurement, we compute the percentate of the variance explained as a function of the number of eigen-vectors used.

### Percentage of variance explained.
![VarExplained.png](report_figures/VarExplained.png)
We see that the top 5 eigen-vectors explain 50% of variance for TMIN, >60% for TOBS and 40% for TMAX.

We conclude that of the three, TOBS is best explained by the top 5 eigenvectors. This is especially true for the first eigen-vector which, by itself, explains about 55% of the variance.

The top 5 eigenvectors explain 14% of the variance for PRCP and 17.5% for SNOW. Both are low values. On the other hand the top 5 eigenvectors explain %80 of the variance for SNWD. This means that these top 5 eigenvectors capture most of the variation in the snow signals. Based on that we will dig deeper into the PCA analysis for snow-depth.

It makes sense that SNWD would be less noisy than SNOW. That is because SNWD is a decaying integral of SNOW and, as such, varies less between days and between the same date on diffferent years.

### Shape of top eigenvectors
![pca_eigen_per_feature.png](report_figures/pca_eigen_per_feature.png)
We can see from above the top eigen vectors have clear shape when the corresponding variance is well explained in previous section. For example, TOBS and SNWD have most clear shapes and their variance are mostly explained, while SNOW and percipitation variances are poorly explained and their eigen vectors have noisy shapes.

## Analysis of correlation between measurements and geography
This section analyzes correlation between different measurements, as well as measurements versus geography(elevation etc)

### Methodology
As previous section shows, only SNWD can be well explained by its top eigenvectors, other measurements are all very noisy and fluctuate a lot during a single year. To measure the correlation between measurements throughout years, we don't care much about the daily variance during the year, but we want to focus on the overal correlation between measurements along the time.

With that, we can smooth the noise by averaging the vector of 365 daily values into monthly, in another word reducing 365 to 12 dimension of vector in each sample.

#### Smooth to monthly
With smoothed dataset, we still have same number of samples, but the vector of measurement values were reduced from 365 dimension to 12.

And below is the new plotting of mean and standard deviation.
![stat_mean_std_monthly.png](report_figures/stat_mean_std_monthly.png)
Plotting of top eigenvectors and variance explained. As we can see below, monthly average measurements now showsclear patterns. 
![pca_eigen_per_feature_monthly.png](report_figures/pca_eigen_per_feature_monthly.png)
And the variance explained is much higher than original dataset.
![VarExplained_monthly.png](report_figures/VarExplained_monthly.png)
Now all the measurements are well represented by their top eigenvectors.

#### Representation with eigen coefficients
To compare weather measurements, it's still not straightforward to compare the 12-d vector from this sample to another. Well, since the 12-d measurement vector of each sample can be well explained by its top eigenvectors, I can simply represent each sample with just coefficients of top eigenvectors. 

So I introduce 3 new features(coeff_0, coeff_1, coeff_2)  to our dataset. I'll just use that for correlation calculation rather than the original 365-d or 12-d vector.

#### filter valid data
Since we want measure correlation between measurements, we need filter valid data with all 6 measurements available for each year and station. With this condition, we only found 249 valid samples out of 10 thousands samples.

### Scattering matrix
Correlation can be visualized by scatter plotting between each 2 features(measurements), and putting together all the scatter plotting of feature pairs gives us below matrix graph. In below matrix, I only included coeff_1(for eigenvector1) because that's the most significant weight which itself alone can represent about 60% variance. Let's check other coefficients further once any interesting pair is found.
![correlation_measures.png](report_figures/correlation_measures.png)

### Findings from above
Even with just 249 valid samples, I can still see some correlation btw different measurements as stated below.
* The 3 temperature measurements TMIN, TMAX, TOBS are positively correlated to each other as expected intuitively.
* SNOW is very positively corrected to SNWD, PRCP, and SNWD and PRCP is slightly correlated.
* TMAX, TMIN, TOBS are negatively correlated to elevation. Notice the TMIN_c1 and elevation shows positive correlation in above graph, but actually TMIN has eigenvector1 below 0, so TMIN itself is negatively correlated to elevation.
* SNOW, SNWD are positively correlated to elevation.

We can further compare each 2 measurements by involving coeff_2, coeff_3, and since each time we only compare 2 of them, we can have more valid data. This will help to confirm these correlations. (I didn't put more effort on this due to lack of time)

### TMAX vs geometry
I picked one measurement TMAX to compare to geometry with more features because this one has most clear correlation with elevation. And we have 2668 valid samples.
![correlation_tmax.png](report_figures/correlation_tmax.png)

### Findings from above graph
* We confirmed that TMAX is highly correlated to elevation(as red circle highlighted). 
* As purple circle indicated, TMAX coeff_2 is somehow correlated to longitude. The corresponding eigenvector2 suggests higher TMAX in summer and lower TMAX in winter, this is seasonal difference in another word. This is correlated to longitude means different locations will see different TMAX variance from season to season.
* TMAX coeff_1 is also correlated to longitude but with 2 distinguished groups, both are negatively correlated. This suggests the stations could be categorized into 2 groups. By checking the map, we can really see the locations are divided by 2 groups, majority on mountain area, the others are far apart in desert area.

## Analysis of snow depth

We choose to analyze the eigen-decomposition for snow-depth because the first 4 eigen-vectors explain 70% of the variance.

First, we graph the mean and the top 4 eigen-vectors.

We observe that the snow season is from early November to the middle of May, where the early of March marks the peak of the snow-depth.
![SNWD_mean_eigs.png](report_figures/SNWD_mean_eigs.png)

Next we look at the eigen-functions. The first eigen-function (eig1) has an opposite but similar shape to the mean function, in another word it's scaled negative of mean. The interpretation of this shape is that eig1 represents the overall amount of snow above/below the mean, but without changing the distribution over time.

**eig2,eig3 and eig4** are similar in the following way. They all oscilate between positive and negative values. In other words, they correspond to changing the distribution of the snow depth over the winter months, but they don't change the total (much).

They can be interpreted as follows:
* **eig2:** less snow in Jan - mid March, more snow in late Mar-Jun.
* **eig3:** slightly less snow in jan, less snow in March, much more snow from Nov to end of year.
* **eig4:** a lot more snow in Jan, sharply droped until middle of March, then hike again through Apr and May.


### Examples of reconstructions

#### samples with minimum and maximum eigen coeffs
With the maximum and minimum values of top 3 eigen coefficients, we can actually get the 6 typical patterns with dramatic difference of SNWD during one year for this area.

Below plotting shows samples with minimum coeff1, coeff2, coeff3, 4 samples each on one row.
![SNWD_grid_min_coeffs.png](report_figures/SNWD_grid_min_coeffs.png)
* On each row, the shape looks quite similar to the according eigen vector, but in mirror opposite(upside down) since coeffs are negative. eg 1st row similar to eigenvector1, 2nd row to eigenvector2.
* The intuitive description of the pattern for each row is just the opposite of according eigenvectors. eg 2nd row has more snow in Jan-mid March, but less snow in late Mar-Jun, as the opposite to eig2 description above.

Below shows similar thing for coeff1-3 with maximum values.
![SNWD_grid_max_coeffs.png](report_figures/SNWD_grid_max_coeffs.png)
* Maximum coeff_1 are still negative values, but they're in much less magnitude compared to minimum values above, at same time noticed residual are comparatively large, means these samples are "not" well explained by eig1 anymore.
* Samples with maximum coeff_2 are large positive values, means they have more snows in April as what we described above for eigenvector2 intuitively.
* Similarly, coeff_3 has large positive values means they have more snows in end of year as what eigenvector3 indicates.

#### Geographical distribution of samples with top coefficients. 
We plotted the locations on map with the total 2(min/max)*3(coeff)*4(samples per measure)=24 samples from above metrics.
As it shows, 24 samples belongs to 11 stations in a very close area.
![dist_snwd_top_coeff.png](report_figures/dist_snwd_top_coeff.png)
Here're 2 observations:
* One same station could show different patterns on different year. One extreme case is station 'USC00047641'(on upper right) which was ever observed in 4 patterns.
* In one year, different stations could have same or different patterns even all the 11 stations are actually very close to each other.

eg In year 1973, out of the 3 samples, 2 stations have quite different SDWD with another one, even they're very close. The difference could be explained by different elevation as annotated on graph.
![dist_snwd_top_coeff_1973.png](report_figures/dist_snwd_top_coeff_1973.png)
In year 1975, all 3 samples have sample pattern, and they turn out to have close elevation which is reasonable.
![dist_snwd_top_coeff_1975.png](report_figures/dist_snwd_top_coeff_1975.png)

### conclusion of snow depth analysis
* This area has significant amount of snow, and snow depth could vary a lot with clear patterns.
* Same station could have quite different snow depth yearly pattern on different years.
* In same year, different locations have dramaticly different snow depth patterns even they're very close in distance.

## Analysis of percipitation
From previous PCA analysis section we can see the percipitation is poorly explained by eigen vectors compared to snow depth(PCRP 17.5% vs SNWD 80% variance explained by top 5 eigen vectors) and PCRP eigen vectors themselves look very fluctuated compared to SNWD's which look very smooth.
It is likely to be hard to find correlations between the amount of rain on the same day in different stations. Because amounts of rain vary a lot between even close locations. It is more reasonable to try to compare whether or not it rained on the same day in different stations. 
![cdf_prcp_daily.png](report_figures/cdf_prcp_daily.png)
As we see from the graph above, in our region it rains in about one fifth of the days which is still fair amount.

### correlation between percipitation across locations
To measure the statistical significance of PCRP correclation across locations, we build up a matrix of all 2 station pairs with each position measuring the log probability that rain days of the 2 stations are overlapped.

![prcp_corr_significance.png](report_figures/prcp_corr_significance.png)
Looking at the histogram of all log probability values above (values are actually negative, we reverse it for easier showing), there're amount of values spreading out to -0.4 which is not very big probability value(about 0.67), which indicates some stations are not purely independent.

![prcp_correlation_matrix.png](report_figures/prcp_correlation_matrix.png)
As the matrix above shows, for each pair of stations, the normalized log probability that the overlap in rain days is random.
But we still see the first 8 stations are highly correlated with each other. 

And this 8 stations are further divided to 2 sub groups with 3 and 5 stations each.


### PCA analysis
To find more correlations we use apply SVD to above matrix. And we can see that the top 10 eigenvectors explain about 80% of the square magnitude of the matrix as below graph.
![prcp_correlation_var_explain.png](report_figures/prcp_correlation_var_explain.png)

When we reorder the rows and columns of the matrix using one of the eigenvectors, the grouping of the 
stations becomes more evident. The 1st eigen vector reorder(upper left matrix) shows about 9 stations are highly correlated, and 2nd eigen vector reorder shows 13 stations correlated.
![prcp_correlation_eig.png](report_figures/prcp_correlation_eig.png)
Actually, when we print out these stations, the 13 stations of 2nd eigen reorder includes all the 9 stations of 1st eigen reorder which again includes all the 8 stations in original matrix. 

### Geographical distribution of top correlated stations based on PRCP.
We ploted the 9 correlated stations(circled) from above 1st eigen reorder on map to check their locations.
* About 5 of them are very closely located near Reno, these 5 stations are actually corresponding to the 5x5 sub block out of the 8x8 highlighted in original matrix.
* We don't see clear geographical similarity of the other 4 stations, could be just coincidence or may be related to factors which we don't know.
![dist_prcp_correlation.png](report_figures/dist_prcp_correlation.png)

## Clustering
As a further step, I tried to categorize the stations according weather measurements, so same group of locations can be described with same climate pattern. The basic idea is to put all measurements of all years as a feature list for each station and use K-Means to cluster them.

So I got a matrix of about 102 stations x 1500 features, each station is a sample with a very long feature list of all measurements of years. If one stations is missing some measurement in some year, a NaN is filled instead, so it's a quite sparse matrix with a lot of missing value.

The most challenge is we don't have many measurement data that is available for all stations at same time. eg. One station has a lot of measurements of year 1970, while another stations has a lot of measurements in year 2000, but there's little overlap between the 2 stations. It doesn't make much sense to compare the 2 stations with data from different years, or from different measurement.

### methodology
Here's some endeavor to overcome above challenge.
* Like what I did for correlation analysis, monthly smoothed data is used here. Monthly average value should be more comparable than daily value which also has a lot of missing days.
* 3 eigen coefficients are used instead of raw monthly data for each measurement, so 3 features instead of 12 for each measurement.
* I dropped the rows(stations) with too many missing features, and then dropped columns(features) that were not available for a lot of stations. 
* Experiments are repeated a lot of times to find a reasonable threshold. I really need to balance the number of rows to drop and number of columns to drop, because too few rows(stations) left won't gain much meaning for this clustering, and dropping too many features will reduce the significance of the result.
* Filled the left missing data with median of same feature.
* So finally, the matrix is reduced from original 105x1500 to 32x150, means I got 32 stations clustered with 150 features.

### Clustering result based on just PRCP
First, I tried to use just one kind of measurement 'PRCP' since it has most count of data. 
I got original matrix of 66 stations x 363 features, and reduced to 18x75 after dropping NaN cells.

Below are the clusters shown on map. Different colors indicates different clusters and size of circle represent elevation.
![cluster_PRCP.png](report_figures/cluster_PRCP.png)
The clustering is kind of making sense since it's aligned with the longitude and elevation as expected.

### cluster result based on all measurements
To be able to cluster more stations, I involved all measurements with good available count across all stations and years.
I got original 105x1500 matrix reduced to 32x150, so I got clustering for 32 stations.
![cluster_all.png](report_figures/cluster_all.png)
It revealed similar grouping which is aligned geography. Noticed the yellow stations are distributed across west to east, but they all have high elevation.

### Conclusion and next step
* Even it's just a small area, the weather could be dramaticlly different in same location from year to year, and from location to location in same year.
* There's certain correlation between different weather measurements and with geography features. 
* Although it's small area, the locations can still be somewhat grouped in to different weather patterns. This is mainly caused by the geography difference which is kind of significant in this mountain area.
* As a further step, we can check the detailed weather pattern for each group, and build a regression model for prediction for any locations in this area without a weather station.